In [6]:
import sys
import os
from glob import glob
sys.path.append("../../marineHeatWaves/")
sys.path.append("../analysis/physiology/")

import marineHeatWaves as mh

from tpc import tpc

import xarray as xr
import numpy as np
import zarr

from shapely import geometry, wkt

import sentinelsat
import satpy
from requests import auth, get
from pyresample.geometry import AreaDefinition

coda_auth = auth.HTTPBasicAuth('tonycan', os.environ['ss_pass'])

api_url='https://coda.eumetsat.int/'

import pandas as pd

import tqdm

from multiprocessing import Pool


import rasterio as rio
from rasterio.plot import show, plotting_extent
from cartopy import crs as ccrs

import seaborn as sns
import matplotlib.pyplot as plt

from functools import partial

from dask.distributed import Client
from dask import delayed
 
from datetime import timedelta

import cloudpickle

import requests
from requests.auth import HTTPBasicAuth

auth = HTTPBasicAuth('tonycan', os.environ['ss_pass'])

import gcsfs
import s3fs 
import boto3

GCP_PROJECT_ID = '170771369993'
OISST_GCP = 'oisst/oisst.zarr'

%matplotlib inline

In [8]:
from functools import partial
import pyproj
from shapely.ops import transform

project = partial(
    pyproj.transform,
    pyproj.Proj(init='epsg:4326'), # source coordinate system
    pyproj.Proj(init='epsg:3857')) # destination coordinate system



# Phytoplankton Isolate Candidates for S3 Validation

Trying to determine here which are the *ideal* isolates to attempt to identify in Sentnel 3 OLCI imagery a signal of MHW impacts esimated via TPCs. 

Possible criteria:

* During a MHW event
* Significant measureable impact in performance due to MHW
* Not close to coast? 

## Load SST + Plankton

In [9]:
plankton = pd.read_csv("../data/Phytoplankton_temperature_growth_rate_dataset_2016_01_29/traits_derived_2016_01_29.csv", engine='python')

plankton = plankton[
    (plankton.habitat == 'marine') & 
    (plankton.curvequal == 'good')
]


plankton.head()

In [10]:
fs = gcsfs.GCSFileSystem(project=GCP_PROJECT_ID, token="../gc-pangeo.json")
oisst = xr.open_zarr(fs.get_mapper(OISST_GCP))
oisst = oisst.assign_coords(lon=(((oisst.lon + 180) % 360) - 180)).sortby('lon')

In [11]:
isolate_performance_perf = xr.open_zarr("../data/isolate_mhw_performance.zarr/isolate_mhw_performance.zarr/")

## Identify MHW statistics for these Isolates. 

In [12]:
mhws_ddf = pd.read_csv("../analysis/isolate_performance_withnegative.csv")

In [13]:
mhws_ddf.peak_date = pd.to_datetime(mhws_ddf.peak_date)
mhws_ddf.start_date = pd.to_datetime(mhws_ddf.start_date)

In [14]:
mhws_ddf

,Unnamed: 0,lat,lon,isolate,mhw,detriment_sum,relative_detriment_mean,detriment_mean,performance_diff_mean,performance_diff_unscaled_mean,...,intensity_mean,duration,start_doy,peak_doy,current_year_sst_mean,start_date,peak_date,perf_det_ratio,latbin,doy_bins
0,0,-74.875,164.625,1,4.0,3.433027,1.303966,0.429128,0.237193,0.078059,...,1.297811,9.0,1,1,-1.202301,1986-12-31,1987-01-01,0.846135,"(-75.026, -59.75]","(0.639, 91.25]"
1,1,-74.875,164.625,1,5.0,6.265167,1.269172,0.417678,0.287250,0.094533,...,1.528319,16.0,21,32,-1.103370,1987-01-21,1987-02-01,0.815611,"(-75.026, -59.75]","(0.639, 91.25]"
2,2,-74.875,164.625,1,11.0,9.585366,1.713322,0.563845,0.035686,0.011744,...,0.187730,18.0,232,244,-1.168743,1988-08-19,1988-08-31,0.979598,"(-75.026, -59.75]","(181.5, 271.75]"
3,3,-74.875,164.625,1,14.0,10.139295,1.711648,0.563294,0.033256,0.010944,...,0.174102,19.0,196,201,-1.181397,1989-07-15,1989-07-20,0.980941,"(-75.026, -59.75]","(181.5, 271.75]"
4,4,-74.875,164.625,1,37.0,6.210158,1.715494,0.564560,0.029738,0.009786,...,0.156030,12.0,215,220,-1.219370,1995-08-03,1995-08-08,0.982961,"(-75.026, -59.75]","(181.5, 271.75]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5294,5294,76.375,-74.625,363,11.0,2.483677,0.563590,0.310460,0.198593,0.109397,...,1.053314,9.0,116,118,-0.199863,1992-04-25,1992-04-27,0.739461,"(61.25, 76.375]","(91.25, 181.5]"
5295,5295,76.375,-74.625,363,15.0,1.662053,0.335243,0.184673,0.198167,0.109163,...,1.308253,10.0,155,160,-0.296657,1995-06-04,1995-06-09,0.626645,"(61.25, 76.375]","(91.25, 181.5]"
5296,5296,76.375,-74.625,363,19.0,2.032843,0.615050,0.338807,0.178253,0.098193,...,0.923265,7.0,4,6,-0.149151,1998-01-04,1998-01-06,0.775308,"(61.25, 76.375]","(0.639, 91.25]"
5297,5297,76.375,-74.625,363,31.0,3.801053,0.530785,0.292389,0.270357,0.148929,...,1.431376,14.0,11,15,0.271288,2003-01-11,2003-01-15,0.662397,"(61.25, 76.375]","(0.639, 91.25]"


# Focus on NE Pacific?

In [15]:
nepac = plankton[
    (plankton['isolation.latitude'].between(40, 80)) &
    (plankton['isolation.longitude'].between(-180, -100))
]
nepac_isolate = nepac.iloc[0]

In [16]:
nepac_isolate

isolate.code                                             240
source                                     Northeast Pacific
isolation.latitude                                        45
isolation.longitude                                     -145
habitat                                               marine
name                                 Pseudo-nitzschia granii
speciesname                          Pseudo-nitzschia granii
former.name.1                                            NaN
former.name.2                                            NaN
strain                                                   NaN
clone                                                    NaN
species                                               granii
genus                                       Pseudo-nitzschia
family                                        Bacillariaceae
order                                          Bacillariales
class                                      Bacillariophyceae
phylum                  

What are the MHWs during the S3 record?

In [17]:
isolate_mhws = mhws_ddf[
    (mhws_ddf.isolate == nepac_isolate['isolate.code']) & 
    (mhws_ddf.start_date >= '2016')
].sort_values('performance_diff_mean', ascending=False)
isolate_mhws

,Unnamed: 0,lat,lon,isolate,mhw,detriment_sum,relative_detriment_mean,detriment_mean,performance_diff_mean,performance_diff_unscaled_mean,...,intensity_mean,duration,start_doy,peak_doy,current_year_sst_mean,start_date,peak_date,perf_det_ratio,latbin,doy_bins
4200,4200,45.125,-144.875,240,52.0,0.613273,0.110637,0.102212,0.145085,0.134037,...,1.731433,7.0,125,127,12.044685,2018-05-05,2018-05-07,0.432335,"(31.0, 46.125]","(91.25, 181.5]"
1353,1353,45.125,-144.875,240,55.0,0.086586,0.003471,0.003207,0.039455,0.036450,...,2.045781,28.0,306,319,12.044685,2018-11-02,2018-11-15,0.159336,"(31.0, 46.125]","(271.75, 362.0]"
4465,4465,45.125,-144.875,240,49.0,0.176060,0.047643,0.044015,-0.047240,-0.043643,...,2.239909,5.0,198,198,11.667212,2016-07-16,2016-07-16,45186.618943,"(31.0, 46.125]","(181.5, 271.75]"
4201,4201,45.125,-144.875,240,54.0,0.932968,0.067324,0.062198,-0.064333,-0.059434,...,2.088545,16.0,284,293,12.044685,2018-10-11,2018-10-20,98467.313719,"(31.0, 46.125]","(271.75, 362.0]"
1638,1638,45.125,-144.875,240,53.0,1.978868,0.125998,0.116404,-0.118718,-0.109678,...,2.732947,18.0,199,213,12.044685,2018-07-18,2018-08-01,17609.652831,"(31.0, 46.125]","(181.5, 271.75]"
2431,2431,45.125,-144.875,240,51.0,8.307739,0.236645,0.218625,-0.195401,-0.180522,...,2.722091,39.0,249,259,11.667212,2016-09-05,2016-09-15,6.400178,"(31.0, 46.125]","(181.5, 271.75]"
2685,2685,45.125,-144.875,240,50.0,6.047693,0.297553,0.274895,-0.240977,-0.222627,...,2.950525,23.0,223,234,11.667212,2016-08-10,2016-08-21,5.241702,"(31.0, 46.125]","(181.5, 271.75]"


Seems like a good idea to test both a MHW with negative and positive performance, but my hunch says we'll see a signal more strongly with a positive effect (perhaps because it's conflated though). 

Either way will start there with that top event (`4200`)



In [18]:
selected_mhw = isolate_mhws.loc[2431]

In [19]:
selected_mhw

Unnamed: 0                                       2431
lat                                            45.125
lon                                          -144.875
isolate                                           240
mhw                                                51
detriment_sum                                 8.30774
relative_detriment_mean                      0.236645
detriment_mean                               0.218625
performance_diff_mean                       -0.195401
performance_diff_unscaled_mean              -0.180522
performance_ratio_mean                       0.794739
intensity_cumulative                          106.162
intensity_mean                                2.72209
duration                                           39
start_doy                                         249
peak_doy                                          259
current_year_sst_mean                         11.6672
start_date                        2016-09-05 00:00:00
peak_date                   

In [20]:
s3_api = sentinelsat.SentinelAPI('tonycan', os.environ['ss_pass'], api_url = 'https://codarep.eumetsat.int/')

In [21]:
isolate_mhw_wkt = wkt.dumps(
    geometry.Polygon.from_bounds(
        *geometry.Point([
            selected_mhw.lon,
            selected_mhw.lat
        ]).buffer(0.5).bounds
    )
)

In [22]:
isolate_mhw_wkt

'POLYGON ((-145.3750000000000000 44.6250000000000000, -145.3750000000000000 45.6250000000000000, -144.3750000000000000 45.6250000000000000, -144.3750000000000000 44.6250000000000000, -145.3750000000000000 44.6250000000000000))'

In [23]:
selected_mhw.start_date, selected_mhw.peak_date

(Timestamp('2016-09-05 00:00:00'), Timestamp('2016-09-15 00:00:00'))

In [24]:
s3_query = s3_api.query(
    area=isolate_mhw_wkt,
    producttype='OL_2_WFR___',
#     date=("20180505", "20180531")
#     date = (isolate_mhws.start_date.min() - timedelta(days=10) , isolate_mhws.start_date.max() + timedelta( days=10)) 
    date = (selected_mhw.start_date - timedelta(days=selected_mhw.duration), selected_mhw.peak_date + timedelta(days=selected_mhw.duration))
)

In [25]:
s3_result = s3_api.to_geodataframe(s3_query)

In [26]:
s3_result.head()

,summary,ecmwf,productlevel,platformname,platformidentifier,format,filename,instrumentname,instrumentshortname,sensoroperationalmode,...,link_icon,title,salinewatercoverpercentage,coastalcoverpercentage,tidalregioncoverpercentage,freshwatercoverpercentage,orbitnumber,relativeorbitnumber,cyclenumber,geometry
2bc7896a-f173-4c22-95b2-d6363faafe01,"Date: 2016-10-23T20:19:32.037Z, Instrument: OL...",ANALYSIS,L2,Sentinel-3,2016-011A,SAFE,S3A_OL_2_WFR____20161023T201932_20161023T20213...,Ocean Land Colour Instrument,OLCI,Earth Observation,...,http://codarep.eumetsat.int/odata/v1/Products(...,S3A_OL_2_WFR____20161023T201932_20161023T20213...,100.0,0.0,0.0,0.0,3566,128,10,"POLYGON ((-158.30700 49.88680, -158.38500 49.3..."
20498d03-5d4d-43e6-8cf9-5435746df013,"Date: 2016-10-21T19:30:40.903Z, Instrument: OL...",ANALYSIS,L2,Sentinel-3,2016-011A,SAFE,S3A_OL_2_WFR____20161021T193041_20161021T19324...,Ocean Land Colour Instrument,OLCI,Earth Observation,...,http://codarep.eumetsat.int/odata/v1/Products(...,S3A_OL_2_WFR____20161021T193041_20161021T19324...,100.0,0.0,0.0,0.0,3537,99,10,"POLYGON ((-146.03000 50.70540, -146.10700 50.1..."
82b615c4-520c-4152-a7e3-f9fb91b10107,"Date: 2016-10-20T19:56:46.056Z, Instrument: OL...",ANALYSIS,L2,Sentinel-3,2016-011A,SAFE,S3A_OL_2_WFR____20161020T195646_20161020T19584...,Ocean Land Colour Instrument,OLCI,Earth Observation,...,http://codarep.eumetsat.int/odata/v1/Products(...,S3A_OL_2_WFR____20161020T195646_20161020T19584...,100.0,0.0,0.0,0.0,3523,85,10,"POLYGON ((-152.52200 51.04950, -152.60000 50.5..."
b13723dd-bc37-4fd9-8f3b-e3ca9cc1450a,"Date: 2016-10-19T20:22:49.977Z, Instrument: OL...",ANALYSIS,L2,Sentinel-3,2016-011A,SAFE,S3A_OL_2_WFR____20161019T202250_20161019T20245...,Ocean Land Colour Instrument,OLCI,Earth Observation,...,http://codarep.eumetsat.int/odata/v1/Products(...,S3A_OL_2_WFR____20161019T202250_20161019T20245...,100.0,0.0,0.0,0.0,3509,71,10,"POLYGON ((-159.00400 51.46550, -159.08200 50.9..."
a840954f-e381-4257-a74c-904e04844c31,"Date: 2016-10-17T19:33:59.026Z, Instrument: OL...",ANALYSIS,L2,Sentinel-3,2016-011A,SAFE,S3A_OL_2_WFR____20161017T193359_20161017T19355...,Ocean Land Colour Instrument,OLCI,Earth Observation,...,http://codarep.eumetsat.int/odata/v1/Products(...,S3A_OL_2_WFR____20161017T193359_20161017T19355...,100.0,0.0,0.0,0.0,3480,42,10,"POLYGON ((-146.72800 52.27050, -146.80600 51.7..."


In [27]:
s3_result.columns

Index(['summary', 'ecmwf', 'productlevel', 'platformname',
       'platformidentifier', 'format', 'filename', 'instrumentname',
       'instrumentshortname', 'sensoroperationalmode', 'mode', 'producttype',
       'timeliness', 'size', 'pduduration', 'orbitdirection', 'relorbitdir',
       'relpassnumber', 'relpassdirection', 'passnumber', 'passdirection',
       'processingname', 'processinglevel', 'procfacilityname',
       'procfacilityorg', 'onlinequalitycheck', 'identifier', 'uuid',
       'beginposition', 'endposition', 'creationdate', 'ingestiondate',
       'collection', 'link', 'link_alternative', 'link_icon', 'title',
       'salinewatercoverpercentage', 'coastalcoverpercentage',
       'tidalregioncoverpercentage', 'freshwatercoverpercentage',
       'orbitnumber', 'relativeorbitnumber', 'cyclenumber', 'geometry'],
      dtype='object')

Before we download anything it seems like the right thing to do to figure out whether we should actually look at this MHW. We'll check out the SST + Performance Regimes @ the isolate location: 

In [ ]:
single_isolate_performance = isolate_performance_perf.sel(isolate=nepac_isolate['isolate.code'])
single_isolate_performance = single_isolate_performance.where(single_isolate_performance.mhw == selected_mhw.mhw, drop=True).compute()

In [ ]:
single_isolate_performance.performance.plot()
single_isolate_performance.performance_clim.plot()
plt.ylim([0, 1])

In [ ]:
single_isolate_performance.performance.plot()
single_isolate_performance.performance_clim.plot()
plt.ylim([0, 1])
[plt.axvline(x=x.beginposition, alpha=0.2, color='red') for _, x in s3_result.iterrows()];


Now we can select images within a Duration/4 buffer around each of the endpoints: 

In [ ]:
selected_mhw.duration

In [ ]:
sensing_start = selected_mhw.start_date - timedelta(days=selected_mhw.duration/4)
sensing_end = selected_mhw.start_date + timedelta(days=selected_mhw.duration + (selected_mhw.duration/4))

In [ ]:
sensing_start, sensing_end

In [ ]:
selected_imagery = s3_result[
    s3_result.beginposition.between(sensing_start, sensing_end)
]
print(len(selected_imagery))


In [3]:
download_location = '/tmp/nepacs3/'

In [ ]:
os.makedirs(download_location, exist_ok=True)

In [ ]:
def download_image(image, path, auth):
    r = requests.get(image.link, auth=auth)
    with open(os.path.join(path, f'{image.title}.zip'), 'wb') as f:
        f.write(r.content)

In [ ]:
delayed_downloads = [delayed(download_image)(x, download_location, auth) for _, x in selected_imagery.iterrows()]

In [ ]:
downloads = client.compute(delayed_downloads, sync=True)

In [ ]:
%%bash -s "$download_location"
ls ${1}*.zip | parallel -t unzip  > /dev/null

In [ ]:
satpy.

In [29]:
## Had to replace _____ with _0000_ for the olci-l2 reader to work...
files = satpy.find_files_and_readers(sensor='olci',
                               base_dir=download_location,
                               reader='olci_l2')


In [4]:
sceneFiles = glob(os.path.join(download_location, "*.SEN3"))

In [ ]:
scenes = [
    satpy.Scene(glob(root+"/*") , reader='olci_l2', ) for root in sceneFiles
]

In [32]:
from satpy.multiscene import timeseries
ms = satpy.MultiScene(scenes)

In [ ]:
for scene in scenes:
    scene.load(['chl_oc4me'])
    print(scene.)

In [33]:
sc = scenes[0]

In [34]:
sc.load(['chl_oc4me'])

In [37]:
sc_resampled = sc.resample(
    sc['chl_oc4me'].attrs['area'].compute_optimal_bb_area(ccrs.Mercator().proj4_params)
)

In [42]:
sc_resampled['chl_oc4me'].compute()

KeyboardInterrupt: 

In [74]:
sc.attrs

{'sensor': {'olci'},
 'start_time': datetime.datetime(2016, 10, 8, 20, 8, 35, 921791),
 'end_time': datetime.datetime(2016, 10, 8, 20, 10, 36)}

In [44]:
sc_resampled.save_dataset('chl_oc4me', writer='geotiff', filename='gtiff2.tif')

In [10]:
def reproject_and_save(img_directory, projection=ccrs.Mercator()):
    _sc = satpy.Scene(glob(img_directory+"/*") , reader='olci_l2', )
    _sc.load(['chl_oc4me'])
    _sc_resampled = _sc.resample(
        _sc['chl_oc4me'].attrs['area'].compute_optimal_bb_area(projection.proj4_params)
    )
    outpath = os.path.join(img_directory, f"chl_oc4me-{_sc.attrs['start_time']:%Y%m%d}.tif")
    _sc_resampled.save_dataset('chl_oc4me', writer='geotiff', filename=outpath, client=False)
    
    


In [ ]:
list(map(reproject_and_save, sceneFiles))

## Which isolates experience the largest mean "mean_performance_ratio"?

In [ ]:
a['performance_ratio_mean_abs'] = a.performance_ratio_mean.abs()

In [ ]:
q_95 = a.performance_ratio_mean.quantile(0.95)
q_05 = a.performance_ratio_mean.quantile(0.05)

In [ ]:
top_isolates_positive_ratio = a.groupby(['isolate']).performance_ratio_mean.median().replace([np.inf, -np.inf], np.nan).sort_values(ascending=False)

In [ ]:
top_isolates_negative_ratio = a.groupby(['isolate']).performance_ratio_mean.median().replace([np.inf, -np.inf], np.nan).sort_values(ascending=True)

In [ ]:
# top_isolates_negative_ratio = a[
#     (a.performance_ratio_mean < 1) &
#     (a.performance_ratio_mean != 0)
# ].groupby('isolate').apply(lambda x: (1 - x['performance_ratio_mean']).max()).replace([np.inf, -np.inf], np.nan).sort_values(ascending=False)

In [ ]:
top_isolates_positive_ratio

In [ ]:
top_isolates_negative_ratio

**Start with Isolates with Above-Clim Performance**

In [ ]:
top_isolate = plankton.set_index('isolate.code').loc[top_isolates_positive_ratio.index[0]]

In [ ]:
top_isolate

In [ ]:
### mhw detections for this isolate: 
isolate_mhws = mhws_ddf[mhws_ddf.isolate == int(top_isolate.name)].compute()

In [ ]:
isolate_mhws

In [ ]:
plt.scatter(isolate_mhws['time'], isolate_mhws['performance']/isolate_mhws['performance_clim'])


In [ ]:
## Find MHW event with highest performance ratio

In [ ]:
isolate_mhws['performance_ratio'] = isolate_mhws.performance / isolate_mhws.performance_clim

In [ ]:
isolate_mhws.groupby('mhw').performance_ratio.median().replace(np.inf, np.nan).dropna().sort_values(ascending=False)

Going to pick 59 because 54 seems anaomalous and short

In [ ]:
selected_mhw = isolate_mhws[isolate_mhws.mhw == 59]

In [ ]:
selected_mhw

In [ ]:
selected_mhw.columns

In [ ]:
selected_sst = oisst.sel(lat = selected_mhw.lat.iloc[0], lon = selected_mhw.lon.iloc[0], time = slice(selected_mhw.time.min(), selected_mhw.time.max()))

In [ ]:
other_year_sst = oisst.sel(
    lat = selected_mhw.lat.iloc[0],
    lon = selected_mhw.lon.iloc[0],
    time = slice(
        selected_mhw.time.min() + np.timedelta64(365, "D"),
        selected_mhw.time.max() + np.timedelta64(365, 'D')
    )
)

In [ ]:
selected_sst

In [ ]:
plt.plot(selected_sst.time, selected_sst.sst)
plt.plot(selected_mhw.time, selected_mhw.clim_seas, color='black')
plt.fill_between(selected_mhw.time, selected_mhw.clim_thresh, selected_sst.sst, color='red', alpha=0.2)
plt.plot(selected_mhw.time, selected_mhw.clim_thresh, color='red')

ax2 = plt.twinx()
ax2.plot(selected_mhw.time, selected_mhw.performance)
ax2.plot(selected_mhw.time, selected_mhw.performance_clim)
ax2.set_ylim([0, 1])


## Get Sentinel 3

In [ ]:
 from datetime import datetime

In [ ]:
s3_api = sentinelsat.SentinelAPI('tonycan', os.environ['ss_pass'], api_url = 'https://codarep.eumetsat.int/')

In [ ]:
isolate_mhw_wkt = wkt.dumps(
    geometry.Point([
        selected_mhw.lon.iloc[0],
        selected_mhw.lat.iloc[0]
    ]).buffer(0.01)
)

In [ ]:
s3_query = s3_api.query(
    area=isolate_mhw_wkt,
    producttype='OL_2_WFR___',
    date = (selected_mhw.time.min() , selected_mhw.time.max()), 
)

In [ ]:
s3_results = s3_api.to_geodataframe(s3_query)

In [ ]:
s3_results.columns

In [ ]:
plt.plot(selected_sst.time, selected_sst.sst)
plt.plot(selected_mhw.time, selected_mhw.clim_seas, color='black')
plt.fill_between(selected_mhw.time, selected_mhw.clim_thresh, selected_sst.sst, color='red', alpha=0.2)
plt.plot(selected_mhw.time, selected_mhw.clim_thresh, color='red')

ax2 = plt.twinx()
ax2.plot(selected_mhw.time, selected_mhw.performance)
ax2.plot(selected_mhw.time, selected_mhw.performance_clim)
ax2.set_ylim([0, 1])
[plt.axvline(i,alpha=0.2) for i in s3_results.beginposition];

In [ ]:
s3_api_new = sentinelsat.SentinelAPI('tonycan', os.environ['ss_pass'], api_url = 'https://coda.eumetsat.int/')

In [ ]:
selected_mhw.time.min() + np.timedelta64(365, 'D')


In [ ]:
s3_query_future = s3_api_new.query(
    area=isolate_mhw_wkt,
    producttype='OL_2_WFR___',
    date = (
        selected_mhw.time.min() + np.timedelta64(365, 'D') ,
        selected_mhw.time.max() + np.timedelta64(365, 'D')
    )
)

In [ ]:
s3_future = s3_api_new.to_geodataframe(s3_query_future)